In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from scipy.sparse import csr_matrix
import numpy as np

# Data Preprocessing

In [2]:
useritemset_trn = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\user_itemset_training.csv', names = ['user', 'itemset'])
useritemset_trn

,user,itemset
0,41813,9149
1,48831,20181
2,43284,23209
3,24833,3285
4,7304,22173
...,...,...
1343763,15391,20491
1343764,43856,21623
1343765,52852,5935
1343766,36640,7656


In [27]:
itemsetitem_trn = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\itemset_item_training.csv', names = ['itemset','item'])
itemsetitem_trn

,itemset,item
0,8571,18506
1,8571,33644
2,8571,32627
3,5725,14855
4,5725,28037
...,...,...
85817,21913,21835
85818,21913,37137
85819,21913,41653
85820,21913,1682


In [28]:
useritem = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\user_item.csv', names = ['user', 'item'])
useritem

,user,item
0,14456,4393
1,14456,11751
2,14456,33757
3,14456,14958
4,14456,16946
...,...,...
2290640,38068,8584
2290641,38068,11744
2290642,38068,30685
2290643,38068,33773


In [29]:
itemsetitem_val_q = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\itemset_item_valid_query.csv', names = ['itemset','item'])
itemsetitem_val_q

,itemset,item
0,6998,28322
1,6998,34217
2,6998,12058
3,8941,10176
4,8941,22364
...,...,...
7792,23768,11696
7793,23768,1738
7794,23768,38048
7795,6019,30882


In [30]:
itemsetitem_val_a = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\itemset_item_valid_answer.csv', names = ['itemset', 'item'])
itemsetitem_val_a

,itemset,item
0,6998,14100
1,8941,37713
2,11717,29703
3,18595,37718
4,23064,42559
...,...,...
2764,396,11105
2765,7525,35558
2766,14719,24249
2767,23768,10764


In [31]:
itemsetitem_test = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\itemset_item_test_query.csv', names = ['itemset','item'])
itemsetitem_test

,itemset,item
0,22934,15950
1,22934,25065
2,22934,33088
3,26086,37355
4,26086,31432
...,...,...
7754,19505,17628
7755,20370,16312
7756,20370,33212
7757,7148,16666


In [32]:
itemsetitem = [itemsetitem_trn, itemsetitem_val_q, itemsetitem_val_a, itemsetitem_test]
itemsetitem = pd.concat(itemsetitem, ignore_index=True)
itemsetitem

,itemset,item
0,8571,18506
1,8571,33644
2,8571,32627
3,5725,14855
4,5725,28037
...,...,...
104142,19505,17628
104143,20370,16312
104144,20370,33212
104145,7148,16666


In [9]:
rating = pd.DataFrame({'rating': [1 for i in range(len(useritemset_trn))]})
useritemset_rating = pd.concat([useritemset_trn, rating], axis=1)
useritemset_rating

,user,itemset,rating
0,41813,9149,1
1,48831,20181,1
2,43284,23209,1
3,24833,3285,1
4,7304,22173,1
...,...,...,...
1343763,15391,20491,1
1343764,43856,21623,1
1343765,52852,5935,1
1343766,36640,7656,1


# Making explicit ratings

In [10]:
for i in tqdm(range(len(useritemset_rating))):
    a = itemsetitem[itemsetitem['itemset'] == useritemset_rating['itemset'][i]]['item']
    b = useritem[useritem['user'] == useritemset_rating['user'][i]]['item']
    useritemset_rating['rating'][i] = useritemset_rating['rating'][i] + len(set(a) & set(b))

  0%|          | 0/1343768 [00:00<?, ?it/s]

In [14]:
address = r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset'
useritemset_rating.to_csv(path_or_buf = address+'\\useritemset_rating.csv')
useritemset_rating

,user,itemset,rating
0,41813,9149,1
1,48831,20181,2
2,43284,23209,1
3,24833,3285,2
4,7304,22173,1
...,...,...,...
1343763,15391,20491,2
1343764,43856,21623,1
1343765,52852,5935,1
1343766,36640,7656,1


# Utility matrix

In [2]:
useritemset_rating = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\useritemset_rating.csv', index_col=0)

In [3]:
min_itemset_ratings = 20

In [4]:
filter_itemset = useritemset_rating['itemset'].value_counts() >= min_itemset_ratings
filter_itemset = filter_itemset[filter_itemset].index.tolist()

In [5]:
ratings_new = useritemset_rating[useritemset_rating['itemset'].isin(filter_itemset)]
print('The original data frame shape:\t{}'.format(useritemset_rating.shape))
print('The new data frame shape:\t{}'.format(ratings_new.shape))

The original data frame shape:	(1343768, 3)
The new data frame shape:	(1179655, 3)


In [6]:
itemset_user_rating = ratings_new.pivot_table('rating', index='itemset', columns='user')
user_itemset_rating = ratings_new.pivot_table('rating', index='user', columns='itemset')

In [7]:
itemset_user_rating.fillna(0, inplace = True)
itemset_user_rating.head()

user,0,1,2,3,4,5,6,7,8,9,...,53887,53888,53889,53890,53891,53892,53893,53894,53895,53896
itemset,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
user_itemset_rating.fillna(0, inplace = True)
user_itemset_rating.head()

itemset,0,1,3,7,10,11,13,15,18,19,...,27678,27680,27681,27683,27684,27685,27687,27688,27691,27692
user,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Itemset-Itemset collaborative filtering

In [10]:
def item_similarity(itemset_user_rating):
    sim = np.corrcoef(itemset_user_rating)
    return sim

In [10]:
itemset_sim = item_similarity(itemset_user_rating)

In [11]:
itemset_sim_df = pd.DataFrame(data=itemset_sim, index=user_itemset_rating.columns, columns=user_itemset_rating.columns)
itemset_sim_df.head(5)

itemset,0,1,3,7,10,11,13,15,18,19,...,27678,27680,27681,27683,27684,27685,27687,27688,27691,27692
itemset,,,,,,,,,,,,,,,,,,,,,
0,1.000000,-0.000627,-0.000468,-0.000544,-0.000562,0.011347,-0.000506,-0.002054,-0.000705,0.022729,...,-0.000763,-0.000605,-0.000475,-0.001469,-0.000705,-0.001095,-0.000455,-0.000475,-0.000458,-0.000932
1,-0.000627,1.000000,-0.000544,-0.000633,-0.000653,-0.001157,-0.000587,0.001185,0.007421,-0.000627,...,-0.000886,-0.000703,-0.000552,-0.001707,-0.000820,-0.001272,-0.000528,-0.000552,-0.000532,-0.001082
3,-0.000468,-0.000544,1.000000,-0.000472,-0.000487,-0.000864,-0.000438,-0.001781,-0.000611,-0.000468,...,-0.000662,-0.000525,-0.000412,-0.001274,-0.000612,-0.000950,-0.000394,-0.000412,-0.000397,-0.000808
7,-0.000544,-0.000633,-0.000472,1.000000,-0.000567,-0.001005,-0.000510,-0.002072,-0.000711,-0.000544,...,-0.000770,-0.000610,-0.000480,-0.001482,-0.000712,-0.001105,-0.000459,-0.000479,-0.000462,-0.000940
10,-0.000562,-0.000653,-0.000487,-0.000567,1.000000,-0.001037,-0.000526,-0.002138,-0.000734,-0.000562,...,-0.000794,-0.000630,-0.000495,-0.001530,-0.000734,-0.001140,-0.000473,-0.000495,-0.000476,-0.000970


In [12]:
def predict_rating(ratings_arr, itemset_sim_arr ): # 평점데이터, item유사도 데이터
    # (평점 x item 유사도) / (item 유사도의 합)
    ratings_pred = ratings_arr.dot(itemset_sim_arr)/ np.array([np.abs(itemset_sim_arr).sum(axis=1)])
    return ratings_pred
    
ratings_pred = predict_rating(user_itemset_rating.values , itemset_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= user_itemset_rating.index,
                                   columns = user_itemset_rating.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(5)

(53897, 13902)


itemset,0,1,3,7,10,11,13,15,18,19,...,27678,27680,27681,27683,27684,27685,27687,27688,27691,27692
user,,,,,,,,,,,,,,,,,,,,,
0,-0.002210,0.000139,0.000309,-0.000457,-0.000368,-0.002140,0.007495,0.004971,-0.000866,0.000431,...,0.000449,-0.000409,-0.002094,0.004175,-0.001474,-0.000371,0.003737,0.001329,0.004236,-0.000355
1,-0.000625,0.001667,0.001278,0.002898,-0.001005,0.000914,0.000741,0.001458,0.002740,0.004829,...,0.000601,0.000314,0.001583,0.000370,-0.001206,0.000755,0.000518,0.001262,-0.000468,0.002818
2,-0.000759,-0.000660,-0.000138,-0.000603,-0.000669,-0.000857,0.001558,-0.000185,-0.000744,0.000367,...,0.001129,0.000137,0.002941,0.000297,-0.000201,0.000237,-0.000154,-0.000701,-0.000642,0.001437
3,-0.003350,-0.001939,-0.000839,-0.001058,0.000242,-0.002424,0.000354,0.003821,-0.000845,0.003464,...,0.001447,0.003085,-0.002653,0.003314,-0.001758,0.003043,0.004968,0.006367,-0.000042,0.000410
4,0.000265,-0.000968,-0.001281,0.000348,0.001676,-0.000446,0.000383,-0.000646,0.000012,-0.001099,...,-0.000046,-0.001197,-0.000529,-0.000559,-0.000924,0.001717,0.000311,-0.001246,-0.001142,-0.001114


In [13]:
ratings_pred_matrix.max(numeric_only=True).max()

2.7693996397769376

In [14]:
ratings_pred_matrix1 = ratings_pred_matrix.applymap(lambda x:0 if x<0.001 else 1)
ratings_pred_matrix1

itemset,0,1,3,7,10,11,13,15,18,19,...,27678,27680,27681,27683,27684,27685,27687,27688,27691,27692
user,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,1,1,0
1,0,1,1,1,0,0,0,1,1,1,...,0,0,1,0,0,0,0,1,0,1
2,0,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1,0,1,...,1,1,0,1,0,1,1,1,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53892,0,0,1,0,0,0,0,1,1,0,...,1,1,0,1,0,0,1,0,1,0
53893,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
53894,1,0,0,0,0,1,1,1,1,1,...,1,1,1,0,0,0,1,0,1,0


# Validation

In [15]:
user_itemset_val_q = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\user_itemset_valid_query.csv', names = ['user','itemset'])
user_itemset_val_q

,user,itemset
0,31914,9939
1,33880,16577
2,9492,15989
3,15847,24682
4,53675,6181
...,...,...
335935,41164,15801
335936,12662,6260
335937,26297,11707
335938,47682,6529


In [16]:
user_itemset_val_a = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\user_itemset_valid_answer.csv', names = ['answer'])
user_itemset_val_a

,answer
0,1
1,0
2,1
3,0
4,1
...,...
335935,0
335936,1
335937,0
335938,1


In [17]:
ans = list(user_itemset_val_a['answer'])
ans

[1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,


In [18]:
acc = 0

for i in tqdm(range(len(user_itemset_val_q))):
    if user_itemset_val_q['itemset'][i] in list(map(int, ratings_pred_matrix1.columns)):
        if ratings_pred_matrix1.loc[user_itemset_val_q['user'][i], user_itemset_val_q['itemset'][i]] == ans[i]:
            acc += 1
        else:
            acc += 0
    else:
        if ans[i] == 1:
            acc += 0
        else:
            acc += 1
    
accuracy = acc/len(user_itemset_val_a) 

  0%|          | 0/335940 [00:00<?, ?it/s]

In [19]:
print(f'accuracy: {accuracy}')

accuracy: 0.7740281002559981


# Train dataset + (True)Validation dataset

In [20]:
useritemset_trn = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\user_itemset_training.csv', names = ['user', 'itemset'])
useritemset_trn

,user,itemset
0,41813,9149
1,48831,20181
2,43284,23209
3,24833,3285
4,7304,22173
...,...,...
1343763,15391,20491
1343764,43856,21623
1343765,52852,5935
1343766,36640,7656


In [21]:
useritemset_val = pd.concat([user_itemset_val_q, user_itemset_val_a], axis=1)
useritemset_val

,user,itemset,answer
0,31914,9939,1
1,33880,16577,0
2,9492,15989,1
3,15847,24682,0
4,53675,6181,1
...,...,...,...
335935,41164,15801,0
335936,12662,6260,1
335937,26297,11707,0
335938,47682,6529,1


In [22]:
useritemset_val_true = useritemset_val[useritemset_val['answer'] == 1]
useritemset_val_true

,user,itemset,answer
0,31914,9939,1
2,9492,15989,1
4,53675,6181,1
6,9267,3725,1
8,46578,2911,1
...,...,...,...
335930,41825,3904,1
335932,8205,5167,1
335934,3595,2520,1
335936,12662,6260,1


In [23]:
useritemset_val_truepair = useritemset_val_true.drop(columns='answer')
useritemset_val_truepair

,user,itemset
0,31914,9939
2,9492,15989
4,53675,6181
6,9267,3725
8,46578,2911
...,...,...
335930,41825,3904
335932,8205,5167
335934,3595,2520
335936,12662,6260


In [24]:
useritemset = pd.concat([useritemset_trn, useritemset_val_truepair], ignore_index = True)
useritemset

,user,itemset
0,41813,9149
1,48831,20181
2,43284,23209
3,24833,3285
4,7304,22173
...,...,...
1511733,41825,3904
1511734,8205,5167
1511735,3595,2520
1511736,12662,6260


In [25]:
rating = pd.DataFrame({'rating': [1 for i in range(len(useritemset))]})
useritemset_rating = pd.concat([useritemset, rating], axis=1)
useritemset_rating

,user,itemset,rating
0,41813,9149,1
1,48831,20181,1
2,43284,23209,1
3,24833,3285,1
4,7304,22173,1
...,...,...,...
1511733,41825,3904,1
1511734,8205,5167,1
1511735,3595,2520,1
1511736,12662,6260,1


In [33]:
for i in tqdm(range(len(useritemset_rating))):
    a = itemsetitem[itemsetitem['itemset'] == useritemset_rating['itemset'][i]]['item']
    b = useritem[useritem['user'] == useritemset_rating['user'][i]]['item']
    useritemset_rating['rating'][i] = useritemset_rating['rating'][i] + len(set(a) & set(b))

  0%|          | 0/1511738 [00:00<?, ?it/s]

In [34]:
address = r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset'
useritemset_rating.to_csv(path_or_buf = address+'\\new_useritemset_rating.csv')

In [2]:
useritemset_rating = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\new_useritemset_rating.csv', index_col=0)

In [3]:
min_itemset_ratings = 20

In [4]:
filter_itemset = useritemset_rating['itemset'].value_counts() >= min_itemset_ratings
filter_itemset = filter_itemset[filter_itemset].index.tolist()

In [5]:
ratings_new = useritemset_rating[useritemset_rating['itemset'].isin(filter_itemset)]
print('The original data frame shape:\t{}'.format(useritemset_rating.shape))
print('The new data frame shape:\t{}'.format(ratings_new.shape))

The original data frame shape:	(1511738, 3)
The new data frame shape:	(1353892, 3)


In [6]:
itemset_user_rating = ratings_new.pivot_table('rating', index='itemset', columns='user')
user_itemset_rating = ratings_new.pivot_table('rating', index='user', columns='itemset')

In [7]:
itemset_user_rating.fillna(0, inplace = True)
itemset_user_rating.head()

user,0,1,2,3,4,5,6,7,8,9,...,53887,53888,53889,53890,53891,53892,53893,53894,53895,53896
itemset,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
user_itemset_rating.fillna(0, inplace = True)
user_itemset_rating.head()

itemset,0,1,3,7,10,11,13,15,18,19,...,27680,27681,27682,27683,27684,27685,27687,27688,27691,27692
user,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
itemset_sim = item_similarity(itemset_user_rating)

In [12]:
itemset_sim_df = pd.DataFrame(data=itemset_sim, index=user_itemset_rating.columns, columns=user_itemset_rating.columns)
itemset_sim_df.head(5)

itemset,0,1,3,7,10,11,13,15,18,19,...,27680,27681,27682,27683,27684,27685,27687,27688,27691,27692
itemset,,,,,,,,,,,,,,,,,,,,,
0,1.000000,-0.000673,-0.000559,-0.000630,-0.000603,0.009795,-0.000575,-0.002395,-0.000819,0.017252,...,-0.000686,-0.000568,-0.000476,-0.001746,-0.000810,-0.001176,-0.000503,-0.000532,-0.000511,-0.001000
1,-0.000673,1.000000,-0.000605,-0.000682,-0.000653,-0.001157,-0.000622,0.000734,0.006394,-0.000667,...,-0.000742,-0.000615,-0.000515,-0.001889,-0.000877,-0.001272,-0.000544,-0.000576,-0.000552,-0.001082
3,-0.000559,-0.000605,1.000000,-0.000566,-0.000542,-0.000961,-0.000516,-0.002151,-0.000736,-0.000554,...,-0.000616,-0.000510,-0.000427,-0.001568,-0.000728,-0.001056,-0.000451,-0.000478,-0.000458,-0.000898
7,-0.000630,-0.000682,-0.000566,1.000000,-0.000611,-0.001083,-0.000582,-0.002425,-0.000829,-0.000624,...,-0.000695,-0.000575,-0.000482,-0.001768,-0.000820,-0.001191,-0.000509,-0.000539,-0.000517,-0.001013
10,-0.000603,-0.000653,-0.000542,-0.000611,1.000000,-0.001037,-0.000557,-0.002322,-0.000794,-0.000598,...,-0.000665,-0.000551,-0.000461,-0.001693,-0.000786,-0.001140,-0.000487,-0.000516,-0.000495,-0.000970


In [13]:
def predict_rating(ratings_arr, itemset_sim_arr ): # 평점데이터, item유사도 데이터
    # (평점 x item 유사도) / (item 유사도의 합)
    ratings_pred = ratings_arr.dot(itemset_sim_arr)/ np.array([np.abs(itemset_sim_arr).sum(axis=1)])
    return ratings_pred
    
ratings_pred = predict_rating(user_itemset_rating.values , itemset_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= user_itemset_rating.index,
                                   columns = user_itemset_rating.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(5)

(53897, 15366)


itemset,0,1,3,7,10,11,13,15,18,19,...,27680,27681,27682,27683,27684,27685,27687,27688,27691,27692
user,,,,,,,,,,,,,,,,,,,,,
0,-0.002367,-0.000245,0.000177,0.000167,-0.000694,-0.002275,0.009980,0.005651,-0.000863,0.001399,...,0.001990,-0.001018,0.002085,0.006194,-0.001727,0.000378,0.005655,0.002565,0.003388,0.000071
1,-0.001133,0.002419,0.002040,0.002237,-0.001227,0.000698,0.000781,0.001519,0.002133,0.004634,...,-0.000188,0.001981,-0.001919,-0.000083,-0.000979,0.000527,-0.000041,0.002200,-0.000858,0.003430
2,-0.000952,-0.000557,0.001083,-0.000886,-0.000946,-0.001200,0.000823,-0.000457,-0.000628,0.000634,...,0.000271,0.001537,-0.000560,0.000087,-0.000570,0.000226,-0.000459,-0.000368,-0.000500,0.001850
3,-0.003298,-0.000795,-0.000591,-0.000243,-0.000150,-0.002528,-0.000241,0.003640,0.001913,0.002203,...,0.002917,-0.002491,-0.002925,0.003335,-0.001915,0.002638,0.002941,0.005054,0.000223,0.000808
4,-0.000206,-0.000685,-0.001349,0.000863,0.001511,-0.000511,0.000006,-0.000998,0.000038,-0.000580,...,-0.000849,-0.000696,-0.000491,-0.000656,-0.001030,0.001201,0.000378,-0.001018,-0.001199,-0.001183


In [14]:
ratings_pred_matrix1 = ratings_pred_matrix.applymap(lambda x:0 if x<0.001 else 1)
ratings_pred_matrix1

itemset,0,1,3,7,10,11,13,15,18,19,...,27680,27681,27682,27683,27684,27685,27687,27688,27691,27692
user,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,1,1,0,1,...,1,0,1,1,0,0,1,1,1,0
1,0,1,1,1,0,0,0,1,1,1,...,0,1,0,0,0,0,0,1,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1,1,1,...,1,0,0,1,0,1,1,1,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53892,0,0,1,0,0,0,0,1,1,0,...,1,0,0,1,0,0,1,0,1,0
53893,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
53894,1,1,0,0,0,0,1,1,1,0,...,1,1,1,0,0,0,1,0,1,0


# Prediction

In [15]:
user_itemset_test_q = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\user_itemset_test_query.csv', names = ['user','itemset'])
user_itemset_test_q

,user,itemset
0,44783,14042
1,23968,3316
2,35829,13546
3,33177,1486
4,10819,5303
...,...,...
335935,34050,10043
335936,18049,18769
335937,10513,20871
335938,15715,22718


In [16]:
user_itemset_test_prediction = []
for i in range(len(user_itemset_test_q)):
    if user_itemset_test_q['itemset'][i] in list(map(int, ratings_pred_matrix1.columns)):
        user_itemset_test_prediction.append(ratings_pred_matrix1.loc[user_itemset_test_q['user'][i], user_itemset_test_q['itemset'][i]])
    else:
        user_itemset_test_prediction.append(0)

user_itemset_test_prediction

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [17]:
print(len(user_itemset_test_prediction))
print(len(user_itemset_test_q))

335940
335940


In [18]:
user_itemset_test_prediction = pd.DataFrame(user_itemset_test_prediction)
user_itemset_test_prediction

,0
0,0
1,1
2,0
3,1
4,0
...,...
335935,1
335936,0
335937,1
335938,1


In [19]:
address = r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset'
user_itemset_test_prediction.to_csv(path_or_buf = address+'\\user_itemset_test_prediction.csv', index=False)